In [ ]:
import pandas as pd
import altair as alt
from decimal import Decimal

In [ ]:
alt.__version__

In [ ]:
data = dict(
    category=["A", "B", "C", "D", "E", "F", "G", "H"], 
    value=[800, 100, 4, 20, 400, 2000, 10, 60] 
)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
def numeric_to_mantissa_and_exponent(value):
    exponent = int(len(str(abs(value)).split(".")[0]) - 1)
    mantissa = float(value * 10 ** -exponent)
    multiplier = int(10 ** exponent)
    
    return mantissa, exponent, multiplier

In [ ]:
df["mantissa"], df["exponent"], df["multiplier"] = zip(*df["value"].map(numeric_to_mantissa_and_exponent))

In [ ]:
df

In [ ]:
def end_x(value):
    return 10 * (value + 1)

In [ ]:
df["end"] = df["exponent"].map(end_x)

In [ ]:
df

In [ ]:
df["end"] = df['end'].cumsum() 

In [ ]:
df

In [ ]:
df["start"] = df["end"].shift(fill_value=0)

In [ ]:
df

In [ ]:
df["middle"] = ((df["end"] - df["start"]) / 2) + df["start"]

In [ ]:
df

In [ ]:
def wsb_chart(data, w=600, h=300):
    base = alt.Chart(data, width=w, height=h)
    
    bar = base.mark_rect().encode(
        x=alt.X('start:Q', axis=alt.Axis(labels=False, title="Category", grid=False, values=data["middle"].to_list())),
        x2=alt.X2("end:Q"),
        y=alt.Y('mantissa:Q', axis=alt.Axis(title="Mantissa"), scale=alt.Scale(domain=[0, 10])),
        color=alt.Color("multiplier:O", title="Magnitude Multiplier", legend=alt.Legend(labelExpr="'× ' + datum.value"), scale=alt.Scale(scheme="reds")),
    
    )
    
    # default `fontSize` = 11
    # default `tickSize` = 5
    # default `labelPadding` = 2
    # default `translate` = 0.5
    text = base.mark_text(align='center', dx=0.5, baseline='middle', fontSize=11).encode(
        x=alt.X('middle:Q'),
        y=alt.value(h + (11/2) + 5 + 2 + 0.5),
        text=alt.Text("category:N")
    )
    
    return bar + text

In [ ]:
wsb_chart(df)